In [ ]:
"""Please keep in mind that this whole document is only
to give a general idea of what to implement. 
These are not functioning codes."""

In [2]:
%pip install "requests==2.28.1"

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
from urllib.parse import urljoin
import requests
sys.path.insert(1, '/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/slang-translator/Connection')
    
from elastic_con import client 
import nltk

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>


In [11]:
import nltk

In [12]:
import spacy
spacy.load("en_core_web_sm")

In [5]:
from gensim.parsing.preprocessing import remove_stopwords

ModuleNotFoundError: No module named 'gensim'

In [9]:
from spacy.matcher import Matcher
from spacy.util import filter_spans

In [5]:
from elasticsearch import Elasticsearch

In [13]:
nlp = spacy.load("en_core_web_sm")

In [24]:
# draft : implementation step
######################################
# translate regular sentence to slang#
######################################

#input_str = "Thanks for taking the time to post. It appears the making of a nation is missing."
input_str = 'He is very hot and sexy '

In [25]:
str_doc = nlp(input_str)

In [26]:
len(str_doc)

6

In [27]:
print(str(list(str_doc.noun_chunks)))

[He]


In [15]:
def noun_phrases_extract(nlp_result):
    # NOTE: the results of spacy for eg
    # .noun_chunks is constituted of elements 
    # rather than simple strings 
    noun_lst = nlp_result.noun_chunks
    return list(noun_lst)

In [16]:
def verb_phrases_extract(nlp_result):
    # under a different scheme
    # acknowledgement: 
    # https://stackoverflow.com/questions/47856247/extract-verb-phrases-using-spacy
    pattern = [[{'POS': 'VERB', 'OP': '?'},{'POS': 'ADV', 'OP': '*'}, # additional wildcard - match any text in between
               {'POS': 'VERB', 'OP': '+'}]]
    print(type(pattern))
    matcher = Matcher(nlp.vocab)
    matcher.add("Verb phrase", pattern)
    matches = matcher(nlp_result)
    spans = [nlp_result[start:end] for _, start, end in matches]
    return filter_spans(spans)
    # preserve the parsed positions of words so that the -ing forms
    # won't get messed up

In [18]:
noun_phrases_extract(nlp("He is very hot and sexy"))

[He]

In [ ]:
""" kopfschmerzen: how to translate "I am really in need of a break" to 
" I rnab" for eg if rnab=really need a break is some kind of legal slang. 
IDK if fuzzy search can do that and I hope urban dic has abbreviations 2"""

In [19]:
# NOTE: not finished.
def phrase_extraction(nlp_result):
    # lemmatize first, extract phrases later
    lst_sentence = []
    for token in nlp_result:
        print(token.pos_)
        word = token.lemma_
        lst_sentence.append(word)
    sentence = " ".join(lst_sentence)
    nlp_result = nlp(sentence)
    print(nlp_result)
    noun_list = noun_phrases_extract(nlp_result)
    verb_list = verb_phrases_extract(nlp_result)
    lst_phrase_temp = noun_list+verb_list
    return lst_phrase_temp

In [32]:

#st1 = 'i will laugh out really loud or i told her never ever to give up'
st1 = "He is very hot and sexy"
s1 = nlp(st1)

In [21]:
phrase_extraction(s1)
# doing an adequate job but can be better

PRON
AUX
ADV
ADJ
CCONJ
ADJ
he be very hot and sexy
<class 'list'>


[he, be]

In [66]:
"I love".split(" ")

['I', 'love']

In [33]:
verb_phrases_extract(s1)
# NOTE: theres room for improvement in accuracy

<class 'list'>


[]

In [21]:
"""To find and return subject of a preprocessed sentence."""
# I wrote the function based on which one may
# omit the subject of a sentence with a possibility. 
# This is not used now, as I still don't
# know how to calculate the weight of such omission.
def subject_find(doc):
    for token in doc:
        if "subj" in token.dep_:
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [22]:
# do slang dictionary query with part of the input sentence
"""returns all possible k-word-shingle for the input sentence.
form: list, [[1-shingle][2-shingle]...[k-shingle]]""" 
def multiple_shingle(doc_str):
    doc = doc_str.split(" ")
    shingle_lst=[]
    for k in range(0,len(doc)):
        shingle_lst_temp = []
        for i in range(len(doc)-k):
            shingle_lst_temp.append(doc[i:k+i+1])
        shingle_lst.append(shingle_lst_temp)
    return shingle_lst

In [34]:
multiple_shingles = multiple_shingle(input_str)

In [410]:
## Remove the extra lengthning of words if present:

## TODO make it general and look into special use cases 
import spacy
import re
nlp = spacy.load("en_core_web_sm")

# def reduce_lengthening(token):
#     if len(token) > 2:
#         pattern = re.compile(r"(.)\1{1,}")
#         return pattern.sub(r"\1", token.text)
#     else:
#         return token.text
def remove_extra_lengthning(sentence):
    words = sentence.split()
    corrected_words = []
    for word in words:
        i = 3
        while i < len(word):
            if word[i] == word[i-1]:
                word = word[:i] + word[i+1:]
            else:
                i += 1
        corrected_words.append(word)
    return ' '.join(corrected_words)


#sentence = "Anyways Titanic is my favorite Movie i cry all the time i see it!"## This will be the informal sentance 
sentence = "It's what I always use for music, movies and books"
sentence = sentence.lower()
sen = remove_extra_lengthning(sentence)
print(sen)

it's what i always use for music, movies and books


In [411]:
stop_words = ['a', 'an', 'the','is','iss','but','in','on','to','my']

# Define a function to remove specific stop words from a sentence
def remove_stop_words(sentence):
    doc = nlp(sentence)
    filtered_sentence = [token.text for token in doc if not token.is_stop or token.text not in stop_words and not token.is_punct]
    return ' '.join(filtered_sentence)

# Test the function on a sample sentence
filtered_sentence = remove_stop_words(sen)
print(filtered_sentence)

it 's what i always use for music , movies and books


In [412]:
## Convert the common abbrevation into expanded form for giving as an input to slang convertor 
## TODO need to generalise this function 
def convert_words(input_string):
    input_string = input_string.replace("do nt", "do not")
    input_string = input_string.replace("wo nt", "would not")
    input_string = input_string.replace("ca nt", "can not")
    input_string = input_string.replace("sha nt", "shall not")
    input_string = input_string.replace("ai nt", "am not")
    input_string = input_string.replace("ai nt't", "am not")
    input_string = input_string.replace("have nt", "have not")
    input_string = input_string.replace("has nt", "has not")
    input_string = input_string.replace("had nt", "had not")
    input_string = input_string.replace("is nt", "is not")
    input_string = input_string.replace("were nt", "were not")
    input_string = input_string.replace("are nt", "are not")
    #input_string = input_string.replace("'s", "it is")
    return input_string


In [413]:

converted_string = convert_words(filtered_sentence)
print(converted_string)

it 's what i always use for music , movies and books


In [414]:
index_name = "slang-demo-json"
request = '''{"query": {"match_all": {}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
#print(results)

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until


In [415]:
def create_search_query(meaning_value):
    query = '''{
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "meaning": "''' + meaning_value + '''"
              }
            }
          ]
        }
      }
    }'''
    return query

#meaning = "he is a sexy and hot person"
results = create_search_query(converted_string)
results = client.search(index=index_name, body=results,size = 100)['hits']['hits']

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


In [416]:
for d in results:
    print(d['_source']['acronym'],'-->',d['_source']['meaning'])

aaf --> always and forever
afs --> always forever and seriously
fwiw --> for what it is worth
iwid --> it is what i do
ouo --> official use only
iiwii --> it is what it is
gfi --> go for it
utfl --> use the force luke
bgm --> background music
mtv --> music television
ialto --> i always love that one
iwaly --> i will always love you
idm --> it does not matter intelligent dance music
4eae --> for ever and ever
wdim --> what does it mean
wtii --> what time is it
idi --> i doubt it
igi --> i get it
ili --> i love it
amv --> anime music video
ebm --> electronic body music
mpa --> music publishers association
vgm --> video game music
vma --> video music awards
wmg --> warner music group
mmi --> me myself and i
aap --> always a pleasure
bfa --> best friends always
lya --> love you always
ainec --> and it is not even close
ttjasi --> take this job and shove it
aisi --> as i see it
aiui --> as i understand it
icbi --> i cannot believe it
iwhi --> i would hit it
anfscd --> and now for something 

In [417]:
meaning = []
acronym  = []
for d in results:
    meaning.append(d['_source']['meaning'])
    acronym.append(d['_source']['acronym'])
slang_dict = dict(zip(acronym,meaning))

In [418]:
def convert_to_slang(input_string):
    for acronym, meaning in slang_dict.items():
        
        input_string = input_string.replace(meaning, acronym)
    return input_string
slang_sen = convert_to_slang(converted_string)
slang_sen

"it 's what i always use for music , movies and books"

In [367]:
# # TODO: ES search function on searching abbreviation of phrases
# """ 
# """
# def search_es_abbreviation(list_words):
    
#     return abbreviation->str

In [ ]:
# """search in abbreviation database based on shingles,
# return slang and the corresponding shingle."""

# def abbreviation_search(shingle_lst):
#     lst_search_result = []
#     for i in range(len(shingle_lst)):
#         list_temp = shingle_lst[i]
#         for j in range(len(list_temp)):
#             word_temp = list_temp[j]
#             search_result = search_es_abbreviation(word_temp)
#             # TODO: make it a fuzzy search
#             if search_result: # when search result non empty
#                 # give list of paired (word, search result)
#                 lst_search_result.append((word_temp, search_result))
#     return lst_search_result

In [ ]:
# """ use Urban dictionary to perform fuzzy search.
# TODO
# NOTE: another possible way is to bring the word to the phrase,
# namely extract keyword from urban dictionary&match it with the phrase."""
# def UD_slang_search(phrase_list):
#     return slang_list
#     # formulation of slang_list: [(slang, word/phrase)]

In [ ]:
# """search in UD database based on phrases,
# return slang and the corresponding phrase."""

# def slang_search(phrase_lst):
#     lst_search_result = []
#     for i in range(len(phrase_lst)):
#         list_temp = phrase_lst[i]
#         for j in range(len(list_temp)):
#             word_temp = list_temp[j]
#             search_result = UD_slang_search(word_temp)
#             # TODO: make it a fuzzy search
#             if search_result: # when search result non empty
#                 # give list of paired (word, search result)
#                 lst_search_result.append((word_temp, search_result))
#     return lst_search_result

In [419]:
 ##TODO need to generalise  use the better emoji dataset if possible 
index_name = "emo31"
results_emo = '''{"query": {"match_all": {}}}'''
results_emo = client.search(index=index_name, body=results_emo,size = 100)['hits']['hits']
#print(results_emo)

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  after removing the cwd from sys.path.


In [420]:
slang_sen

"it 's what i always use for music , movies and books"

In [421]:
def create_search_query(meaning_value):
    query = '''{
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "meaning": "''' + meaning_value + '''"
              }
            }
          ]
        }
      }
    }'''
    return query

#meaning = "he is a sexy and hot person"
results_emo = create_search_query(slang_sen)
results_emo = client.search(index=index_name, body=results_emo,size = 100)['hits']['hits']


/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


In [422]:
for d in results_emo:
    print(d['_source']['emoji'],'-->',d['_source']['meaning'])

😦 --> ['frowning_face_with_open_mouth', 'face', 'aw', 'what']
📚 --> ['books', 'literature', 'library', 'study']
😦 --> ['frowning_face_with_open_mouth', 'face', 'aw', 'what']
📚 --> ['books', 'literature', 'library', 'study']
😖 --> ['confounded_face', 'face', 'confused', 'sick', 'unwell', 'oops', ':S']
😖 --> ['confounded_face', 'face', 'confused', 'sick', 'unwell', 'oops', ':S']
🪗 --> ['accordion', 'music']
🪗 --> ['accordion', 'music']
📯 --> ['postal_horn', 'instrument', 'music']
🎶 --> ['musical_notes', 'music', 'score']
🎸 --> ['guitar', 'music', 'instrument']
🎺 --> ['trumpet', 'music', 'brass']
🪕 --> ['banjo', 'music', 'instructment']
🪘 --> ['long drum', 'music']
📯 --> ['postal_horn', 'instrument', 'music']
🎶 --> ['musical_notes', 'music', 'score']
🎸 --> ['guitar', 'music', 'instrument']
🎺 --> ['trumpet', 'music', 'brass']
🪕 --> ['banjo', 'music', 'instructment']
🪘 --> ['long drum', 'music']
🎧 --> ['headphone', 'music', 'score', 'gadgets']
🎧 --> ['headphone', 'music', 'score', 'gadgets'

In [423]:
meaning = []
emoji  = []
for d in results_emo:
    meaning.append(d['_source']['meaning'])
    emoji.append(d['_source']['emoji'])
emo_dict = dict(zip(emoji,meaning))

In [424]:
def replace_with_emoji(sentence):
    words = sentence.split()
    print(words)
    emojified = [next((emoji for emoji, meanings in emo_dict.items() if word in meanings), word) for word in words]
    return " ".join(emojified)
print(replace_with_emoji(slang_sen))

['it', "'s", 'what', 'i', 'always', 'use', 'for', 'music', ',', 'movies', 'and', 'books']
it 's 😦 i always use for 🪗 , movies and 📚


In [46]:
meaning = results_emo[0]['_source']['meaning'][0]
emoji = results_emo[0]['_source']['emoji']
emo_dict = {emoji:meaning}

In [47]:
emo_dict

{'🧑': 'person'}

In [48]:
def convert_to_emoji(input_string):
    for emoji, meaning in emo_dict.items():
        
        input_string = input_string.replace(meaning, emoji)
    return input_string
slang_sen = 'idk, but he is way hot'
slang_sen1 = 'He is a  sexy man'
slang_sen = convert_to_emoji(slang_sen1)
slang_sen1

'He is a  sexy man'

In [51]:
input_string = 'He is very hot person'
for emoji, meaning in emo_dict.items():
        
    input_string = input_string.replace(meaning, emoji)
print(meaning)

person


In [ ]:
# TODO: ES search in emoji database and return 
# list of corresponding emojis
def emoji_search_es(word):
    return emoji_list->list
    # return a list of emoji searched
    # can be UTF-8 encoded or numbered according to order in dataset

In [34]:
"""remove stop words,
perform a word association based on twitter corpus.
perform ES query based on associated words.
return: corresponding word and associated emoji as list of tuple."""


def emoji_search(input_str):
    emoji_search_results = []
    en_stops = nltk.corpus.stopwords.words("english")
    word_list = [x.strip() for x in input_str.split(" ")]
    model = Word2Vec.load('model.bin')
    for word in word_list:
        emoji_search_temp = []
        if word not in en_stops:
            # perform association
            # perform emoji search
            list_of_emoji_original = emoji_search_es([word])
            emoji_search_temp = emoji_search_temp + list_of_emoji_original
            if not len(list_of_emoji_original):
                # if the origial word cannot be found
                association_list = model.wv.most_similar(word)
                for word_associated in association_list:
                # store in list_of_emoji
                    list_of_emoji = emoji_search_es(word_associated)
                    emoji_search_temp = emoji_search_temp + list_of_emoji
        emoji_search_results.append((word, emoji_search_temp))
    return emoji_search_results
    # TODO: can also based on api to reversedictionary.com
    # which gives associated words to noun phrases: 
    # 1. extract noun phrases with nlp().noun_chunks,2.search
    # the phrase up in reversedictionary.com, and
    # 3.perform query with ES in emoji dataset with obtained 
    # set of words.

In [ ]:
def emoji_search_alternative(word_lst):
    emoji_search_results = []
    for word in word_lst:
        emoji_search_temp = []
            # perform association
        association_list = model.wv.most_similar(word)
            # perform emoji search
        for word_associated in association_list:
            # store in list_of_emoji
            list_of_emoji = emoji_search_es(word_associated)
        # TODO: return word_associated with associated emoji
            emoji_search_temp = emoji_search_temp + list_of_emoji
        emoji_search_results.append((word, emoji_search_temp))
    return emoji_search_results

In [ ]:
"""The general function that would 
translate input to sentence with slang&emoji.
return&print translated sentence.
To be continued.
"""
def translation_style(input_str):
    # for each of the words perform emoji search first
    # if we want the precise results of emoji association,
    # it would be best to not perform association based on trivial words
    # -> STOP WORD REMOVAL
    emoji_list = emoji_search(input_str)
    # completely word based so we might just 
    # append the emoji after words.
    # another way is to only perform association 
    # when you encounter a certain grammartical structure
    # for eg noun phrase&verb phrase. 
    str_doc = nlp(input_str)
    word_list = []
    for word in str_doc:
        if word.pos_ == "VERB" or word.pos_ == "NOUN" or word.pos_ == "ADJ":
            word_list.append(word)
    emoji_list_alternative = emoji_search_alternative(word_list)
    # need to note the order of the words since same word
    # can appear multiple times as multiple POS
    phrase_list = phrase_extraction(str_doc)
    
    shingle_lst = multiple_shingle(input_str)
    # the problem here is that after translating input
    # to slangs the word formulation would differ.
    # ->should make sure which part translates to which
    abbrev_list = abbreviation_search(shingle_lst)
    # search for further slangs with the original sentence
    # and if the slang results overlap with the abbreviations
    # for eg got 2 slangs for same word
    # then use any one with equal probability
    slang_list = slang_search(phrase_list)
    # TODO: get the results together:
    # append the emojis->
    # see if the abbrev list clashes with the UD one->
    # decide the final list->
    # substitute words with slangs
    return slang_sentence


    
    
    